In [0]:
# Importación de librerias
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import re
from unicodedata import normalize
from itertools import chain
import handyspark as hdy
import matplotlib.pyplot as plt
import seaborn as sns
from sparkdl.xgboost import XgboostClassifier

from pyspark.sql.types import StringType
from pyspark.ml.linalg import Vectors
from pyspark.ml.functions import vector_to_array
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoder, VectorSizeHint
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, LongType, BooleanType, DoubleType, TimestampType
from pyspark.sql.functions import desc, length, col,isnan,when,count
import pyspark.sql.functions as F

import mlflow
import mlflow.spark

from hyperopt import fmin, tpe, hp, SparkTrials, Trials, STATUS_OK
from hyperopt.pyll import scope
from math import exp
from mlflow.models.signature import infer_signature
from sklearn.metrics import roc_auc_score, balanced_accuracy_score, confusion_matrix, classification_report, confusion_matrix

# 1) Guardado de datos necesarios para entrenamiento de modelos, excepto DescripcionPeticion

In [0]:
# Solamente se subirá el 20% de los datos necesarios para replicar el entrenamiento de los datos, por lo que se parte del conjunto de datos cruzados y limpios luego de su procesamiento por los notebooks "1_Importacion y cruce" y "2_Limpieza y creacion de variables"

# Leer archivo parquet donde se encuentra la base limpia
file = "/mnt/basecruzada/baselimpia.parquet"
DF_modelo = spark.read.parquet(file)

# Se dejan solo registros de mayo 2020 para atrás, ya que fueron estos los utilizados en los modelos de ML 
print((DF_modelo.count(), len(DF_modelo.columns)))
DF_modelo = DF_modelo.filter((col("AnoRegistroPeticion") != "2021"))
print((DF_modelo.count(), len(DF_modelo.columns)))
DF_modelo = DF_modelo.filter((col("AnoRegistroPeticion") != "2020") | ~(col("MesRegistroPeticion").isin(["Jun", "Jul", "Aug", "Sep", "Oct","Nov","Dec"])))
print((DF_modelo.count(), len(DF_modelo.columns)))

# Seleccionar solo columnas necesarias para entrenamiento de modelo
columnas_mantener = ['AnoRegistroPeticion', 'horaRegistroPeticion', 'minutoRegistroPeticion', 'X', 'Y', 'porc_etnica_2018', 'porc_indigena_2018', 'tasa_vip_total', 'tasa_vif_total', 'TasaFecunidad_15a19', 'porc_rural', 'vict_delsexual_total', 'EdadAfectado_Anios_Imputada', 'MesRegistroPeticion', 'MotivoPeticion', "TipoPeticion", 'CanalRecepcion','TipoDocumentoPeticionario', 'EdadPeticionario', 'SexoPeticionario', "DetalleZonaPeticionario", 'TipoDocumentoAfectado', 'SexoAfectado', "DetalleZonAfectado", 'GrupoEtnicoAfectado', 'PresentaDiscapacidadAfectado', 'CondicionDesplazamientoAfectado', 'PaisAfectado_5', "CategorizacionPeticionario", "SinSuficienteInfo_Afectado", 'DescripcionPeticion', 'VAR_OBJETIVO']

DF_modelo = DF_modelo.select(columnas_mantener)

(1163243, 118)
(1088405, 118)
(982085, 118)

In [0]:
# Se realiza un muestreo estratificado del 20% de los registros. La estratificación se realiza por categoría de la variable objetivo, para que de para cada categoría se tome el 20% de los datos
muestra = DF_modelo.sampleBy("VAR_OBJETIVO", fractions={"verdadera_nopard": 0.2, "falsa": 0.2, "sindefinir_fallida":0.2, "verdadera_pard_noinst":0.2, "verdadera_pard_inst":0.2}, seed=5)
print((muestra.count(), len(muestra.columns)))

(196858, 32)

In [0]:
# Se separa la columna de "DescripcionPeticion", ya que esta contiene información sensible y no puede ser difundida tal como está. Todas las otras columnas contienen información que no permite la identificación particular de una persona
muestra_sinDescripcion = muestra.drop("DescripcionPeticion")
muestra_Descripcion = muestra.select("DescripcionPeticion")

In [0]:
# Los datos sin la Descripción se pueden exportar directamente
file_location = "/mnt/basecruzada/Muestra_DatosAbiertos/datos_modelo_sinDescripcion"
muestra_sinDescripcion.coalesce(1).write.mode("overwrite").option("header", "true").mode("overwrite").parquet(file_location)

# 2) Guardado de features/variables creadas a partir del texto de DescripcionPeticion
Los datos de descripción no se exportarán directamente, sino que se transforman en features mediante distintos métodos

## 2.1) Guardado mediante conteo de palabras más frecuentes
El primero de los métodos en el que se extraen variables a partir del texto es el utilizado para el entrenamiento de redes neuronales, que cuenta el número de veces que en cada descripción aparecen las 200 palabras más frecuentes en todo el corpus

In [0]:
# Esta transformación fue guardada mediante MLflow como un pipeline de Sklearn, por lo que es necesario importarlo
saved_text_pipeline = 'runs:/38239dc6bb4349d1b563ec5aa5928002/nn_text_pipeline'
text_pipeline = mlflow.sklearn.load_model(saved_text_pipeline)

# La columna de texto se debe transformar a Pandas
muestra_Descripcion_pd = muestra_Descripcion.toPandas()

# Debido a que hay algunas descripciones que están vacías, estas deben llenar con el texto "(Vacío)"
muestra_Descripcion_pd = muestra_Descripcion_pd.fillna("")

# Hacer transformación de texto en padded_sequence con pipeline importado
padded_sequence = text_pipeline.transform(muestra_Descripcion_pd["DescripcionPeticion"])

# El resultado de la línea anterior es un array de Numpy de tamaño (196858, 200). Para poder guardarlos se transformarán a un Dataframe de Pandas, para lo que primero se crea una lista con el nombre de las columnas del dataframe que será [tokenizer_0, tokenizer_1, ... , tokenizer_199]
tokenizer_columns = ["textfeature_"+ str(i) for i in range(200)]

# Ahora se crea el Dataframe
final_tokenizer_pd = pd.DataFrame(data=padded_sequence, columns=tokenizer_columns)

# Se transforma este Pandas Dataframe a un Spark Dataframe para guardar estos datos de forma similar que los otros datos en formato parquet
final_tokenizer = spark.createDataFrame(final_tokenizer_pd)

# Se guarda como archivo Parquet en el Storage Account
file_location = "/mnt/basecruzada/Muestra_DatosAbiertos/datos_modelo_Descripcion_tokenizer"
final_tokenizer.coalesce(1).write.mode("overwrite").option("header", "true").mode("overwrite").parquet(file_location)

## 2.2) Guardado mediante TD-IDF
Ahora se exportarán las variables extraídas del texto mediante el método de TD-IDF, aplicado en el notebook "3_PipelineSpark_TFIDF_Embedding" y que es el utilizado para el entrenamiento de Gradient Boosted Trees con XGBoost en los notebooks "4.3_ML_XGBoost_TFIDF_Weights" y '4.4_ML_XGBoost_TFIDF_NoWeights'

In [0]:
# Esta transformación fue guardada mediante MLflow como un pipeline de SparkML, por lo que es necesario importarlo
# Se carga el pipeline entrenado en el cuadernillo "3_PipelineSpark_TFIDF_Embedding", el correspondiente al TF-IDF que genera 300 features/variables 
pipeline_tfidf = mlflow.spark.load_model("runs:/02cee8d29a894ebc86ace1eb965e0d12/tfidf_text_featuriser")

# Se transformar el texto en variables haciendo transform con el pipeline importado. Solo se mantiene la columna llamada "tfidf"
tfidf_muestra_Descripcion = pipeline_tfidf.transform(muestra_Descripcion).select("tfidf")

# La columna "tfidf" contiene las variables creadas a partir del texto como un vector disperso de largo 300, se deben transformar a distintas columnas para guardarlo como archivo parquet
tfidf_muestra_Descripcion = tfidf_muestra_Descripcion.withColumn("tfidf", vector_to_array("tfidf")).select([col("tfidf")[i] for i in range(300)])

tfidf_muestra_Descripcion.cache()
print(tfidf_muestra_Descripcion.count())

# Se guarda como archivo Parquet en el Storage Account
file_location = "/mnt/basecruzada/Muestra_DatosAbiertos/datos_modelo_Descripcion_tfidf"
tfidf_muestra_Descripcion.coalesce(1).write.mode("overwrite").option("header", "true").mode("overwrite").parquet(file_location)

2021/10/04 03:14:27 INFO mlflow.spark: 'runs:/02cee8d29a894ebc86ace1eb965e0d12/tfidf_text_featuriser' resolved as 'dbfs:/databricks/mlflow-tracking/3030803122807175/02cee8d29a894ebc86ace1eb965e0d12/artifacts/tfidf_text_featuriser'
2021/10/04 03:14:27 INFO mlflow.spark: File 'dbfs:/databricks/mlflow-tracking/3030803122807175/02cee8d29a894ebc86ace1eb965e0d12/artifacts/tfidf_text_featuriser/sparkml' not found on DFS. Will attempt to upload the file.
2021/10/04 03:15:03 INFO mlflow.spark: Copied SparkML model to /tmp/mlflow/a19c9ce6-f460-49d5-8cf9-d4a91e275581
196858

## 2.3) Guardado mediante Word Embeddings
Ahora se exportarán las variables extraídas del texto mediante el método de Word Embeddings, que fue aplicado en el notebook "3_PipelineSpark_TFIDF_Embedding" y que es el utilizado para el entrenamiento de Gradient Boosted Trees con XGBoost en el notebook "4.2_ML_XGBoost_Embedding_Weights"

In [0]:
# Esta transformación fue guardada mediante MLflow como un pipeline de SparkML, por lo que es necesario importarlo
# Se carga el pipeline entrenado en el cuadernillo "3_PipelineSpark_TFIDF_Embedding", el correspondiente al Word Embedding que genera 100 features/variables 
pipeline_wemb = mlflow.spark.load_model("runs:/abd6d3995aaf4c65b2577b73a9be62cf/word_embedding_text_featuriser")

# Se transformar el texto en variables haciendo transform con el pipeline importado. Solo se mantiene la columna llamada "finished_embeddings_vector"
wemb_muestra_Descripcion = pipeline_wemb.transform(muestra_Descripcion).select("finished_embeddings_vector")

# La columna "finished_embeddings_vector" contiene las variables creadas a partir del texto como un vector disperso de largo 100, se deben transformar a distintas columnas para guardarlo como archivo parquet
wemb_muestra_Descripcion = wemb_muestra_Descripcion.withColumn("finished_embeddings_vector", vector_to_array("finished_embeddings_vector")).select([col("finished_embeddings_vector")[i] for i in range(100)])

wemb_muestra_Descripcion.cache()
print(wemb_muestra_Descripcion.count())

# Se guarda como archivo Parquet en el Storage Account
file_location = "/mnt/basecruzada/Muestra_DatosAbiertos/datos_modelo_Descripcion_wordembeddings"
wemb_muestra_Descripcion.coalesce(1).write.mode("overwrite").option("header", "true").mode("overwrite").parquet(file_location)

2021/10/04 03:25:12 INFO mlflow.spark: 'runs:/abd6d3995aaf4c65b2577b73a9be62cf/word_embedding_text_featuriser' resolved as 'dbfs:/databricks/mlflow-tracking/3030803122807175/abd6d3995aaf4c65b2577b73a9be62cf/artifacts/word_embedding_text_featuriser'
2021/10/04 03:25:13 INFO mlflow.spark: File 'dbfs:/databricks/mlflow-tracking/3030803122807175/abd6d3995aaf4c65b2577b73a9be62cf/artifacts/word_embedding_text_featuriser/sparkml' not found on DFS. Will attempt to upload the file.
2021/10/04 03:25:59 INFO mlflow.spark: Copied SparkML model to /tmp/mlflow/ba817413-c167-4e44-bfb0-b0765806982e
196858